In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import typing
import types
from typing import Annotated, Dict, List, Optional
from __future__ import annotations


from pydantic import BaseModel, PlainSerializer, model_validator, Field, BeforeValidator

from qat.model.component import Component, ComponentId
from qat.model.refs import Ref, RefDict, RefList
from qat.model.autopopulate import AutoPopulate

from typing import NewType
import uuid

from pydantic import ValidationError

from copy import deepcopy

In [3]:
def make_refdict(*items: list[Component]):
    return {i.to_component_id() : i for i in items}


In [ ]:
class A(Component):
    pass

class B(Component):
    x: int
    p: Ref[A]

a=A()
B(x=5,p=a)

In [5]:


class C(Component):
    z: RefList[A]
    rd: RefDict[A]
    p: Ref[A]
    j: Ref[A]
    q: Optional[int]
    y: int

a = A()
b = A()
c = C(rd=make_refdict(a,b), z=[ComponentId(),ComponentId()], p=ComponentId(), j=a, q=None, y=3)




In [ ]:
print(c.model_dump_json(indent=2))

In [7]:
class A(Component):
    pass

class C(Component):
    x: RefDict[A]
    y: None | int
    z: RefList[A]
    p: Ref[A]
    j: Ref[A]
    q: Optional[int]

a = A(id='test1')
b = A(id='test2')
c = C(x=make_refdict(a,b), y=None, z=[ComponentId(),ComponentId()], p=ComponentId(), j=a, q=None, id="test")

expected = {'x': {'type': 'RefDict', 'populated': True},
 'z': {'type': 'RefList', 'populated': False},
 'p': {'type': 'Ref', 'populated': False},
'j': {'type': 'Ref', 'populated': True}}

assert c._ref_fields == expected

In [ ]:
a.model_dump_id()

In [9]:
class At(Component):
    x: int
    
class Bt(Component):
    x: int
    s: str
    As: RefDict[At]
    Ds: RefList[Dt]

class Ct(Component):
    x: int
    As: RefDict[At]
    Bs: RefList[Bt]
    someB: Ref[Bt]

class Dt(Component):
    x: int
    Cs: RefDict[Ct]

In [10]:
class Outer(AutoPopulate):
    A: list[At]
    B: list[Bt]
    C: list[Ct]
    D: list[Dt]

In [11]:

def make_Outer(count=10, connections=3, seed=42):
     rng = np.random.default_rng(seed)
     pick = lambda L, size=3: make_refdict(*rng.choice(L, size=size))
     A = [At(x=rng.integers(100)) for _ in range(count)]
     B = [Bt(x=rng.integers(100), As=pick(A,3), s=f'blah{rng.integers(100)}', Ds=[]) for i in range(count)]
     C = [Ct(x=rng.integers(100), As=pick(A,3), Bs=list(pick(B,3).values()), someB=list(pick(B,1).values())[0]) for i in range(count)]
     D = [Dt(x=rng.integers(100), Cs=pick(C,3)) for i in range(count)]

     for bt in B:
          bt.Ds = list(pick(D,3).values())

     return Outer(A=A, B=B,C=C,D=D, id='outer')

In [ ]:
O1 = make_Outer(seed=43)
O1.B[0].Ds[0].x

In [ ]:
print(O1.model_dump_json(indent=2))

In [ ]:
O2 = Outer(**O1.model_dump())
O2.B[0].Ds[0].x

In [ ]:
O1.model_dump()['D'][0]['Cs']

In [16]:
O3 = make_Outer(seed=6353234234)
for idx in range(100):
    seed = np.random.randint(10_000)
    O1 = make_Outer(seed=seed)
    blob = O1.model_dump()
    O2 = Outer(**blob)
    blob2 = O2.model_dump()

    assert O1._deepequals(O2)
    assert not O1._deepequals(O3)

    O3 = deepcopy(O1)

    assert O3._deepequals(O1)
    O3.C[3].x = -1

    assert not O3._deepequals(O1)


In [17]:
for idx in range(100):
    O1 = make_Outer(seed=idx)
    blob = O1.model_dump()
    O2 = Outer(**blob)
    blob2 = O2.model_dump()

    assert blob == blob2

In [18]:
for idx in range(10):
    O1 = make_Outer(count=50, seed=idx)
    blob = O1.model_dump()
    O2 = Outer(**blob)

    assert O1._deepequals(O2)

In [19]:
O1 = make_Outer(count=50, seed=idx)
O2 = deepcopy(O2)
O2.C[3].x = -1

In [ ]:
%%timeit
O1 == O2

In [ ]:
%%timeit
blob = O1.model_dump()
O2 = Outer(**blob)
blob2 = O2.model_dump()

assert blob == blob2